<a href="https://colab.research.google.com/github/Mathis1993/DCGAN/blob/master/DCGAN_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to this small workshop on GANs. Before we really start, we have to organize our hardware (thanks, Google!) and access to our data.

## Choose Hardware Accelerator

To run on GPU, from the menu, select
* Runtime -> Change Runtime Type --> Hardware Accelerator --> GPU


## Mount Google Drive

Before starting, we want to mount our Google Drive to have access to our data in colab.

This is where our data is located. The folder should be shared with everybody. If that's not the case, the data can be found [here](https://drive.google.com/open?id=19zewI-wxU-EACyo5yWyrIErzlj-sX-1j) to be added to your drive. The full dataset can be found [here](https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg).

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Next, specify the directory holding our data.

In [0]:
path =  "/content/drive/My Drive/"

Now, we can get started with our Deep Convolutional Generative Adversarial Network (DCGAN).

## DCGAN

We want to create a network through which we are able to generate images of faces that look as real as possible. Or, put differently, we want a network to learn the distribution of face data, to be able to sample new data (new faces) from that distribution.

This tutorial is mainly inspired by [this official pytorch tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) by Nathan Inkawhich.

### Imports

In [0]:
from __future__ import print_function
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML

#Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed) #set randoms' random seed
torch.manual_seed(manualSeed) #set pytorch's random seed

### Parameters

The hyperparameters' values (set here and in the model section) are best practice recommendations from [this paper](https://arxiv.org/pdf/1511.06434.pdf).

In [0]:
#Root directory for dataset
dataroot = path + "data_workshop"

#Number of workers for dataloader
workers = 8

#size using a transformer.
image_size = 64

#Batch size during training
batch_size = 128

#Number of training epochs
num_epochs = 30

#Learning rate for optimizers
lr = 0.0002

#Beta1 hyperparam for Adam optimizers
beta1 = 0.5

#Whether to use GPU
gpu = True

## GPU device

In order to use the GPU Google is offering to us, we create a pytorch device. When creating a tensor or a model later, we can do that on this device or push the object to this device. Like that, all the calculations can easily run on the GPU.

In [0]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and gpu) else "cpu")
print(device)

### Data

Firts, we specify the structure of our dataset. We set our image folder up in way that we can use it with Pytorch's ImageFolder Dataset-class: In a directory, we have one or more subdirectories containing the images.
We can specify transforms here, that are called when the data is loaded. Here, we resize the images to have 64x64 pixels, we declare them to be tensors and we normalize the range of their values (which orignially are at [0,1] to [-1,1], as this is the range the generator will output later.

In [0]:
#Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

Next, we can feed the dataset into an instance of the DataLoader-class, specifying the batch_size, if batches should be drawn randomly and how many cores should work on retrieving batches in parallel.

In [0]:
#Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

How many batches are loaded? It might take a while until all files from the shared data folder "arrived" in our drive. We have 5000 images, so with a batch size of 128 that should sum up to around 40 batches. If it's less, rerun the dataloader after some time.

In [0]:
len(dataloader)

Our dataloader is iterable, so we can call next() on it to retrieve a batch of randomly selected images. We want to plot 64 of these in a grid (conveniently done with pytorch's make_grid-function) to take a peek at our data. Note that we have to swap the tensors' axes (that hold the image data) for plotting. Pytorch usually represents image data as (n_channels x height x width) while numpy does as (height x width x n_channels).
Also, as currently our images are in a range of [-1,1], which pyplot can't work with, we need to normalize them to [0,1] again.

In [0]:
#Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
#dataloader gives tuple of (data-tensor, target-tensor) with targets defaulting to 0 (as we only used one subfolder)
#make grid on gpu, then push it to cpu so numpy can use it
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0))) #normalize from [-1,1] to [0,1] again

### Weight Initialization

We want to initialize the weights of convolutional and batch-norm layers with values from specific normal distributions. We can do that when initializing our models with this function.

In [0]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### Models
We have two physical models:
1. The **discriminator**: A regular CNN taking a batch of images as an input, and classifying for each image if it believes it to be real or not. It outputs the probability of an image being real (in its opinion).

2. The **generator**: Into this model, we feed a vector of random numbers (drawn from the standard normal distribution). These are transformed into a 3x64x64 color image. We want this image to look real (we want the generator to capture the images' distribution).

#### Discriminator

The discriminator is a classical classifier. We can imagine its layers processing the 3x64x64 input images to become a prediction ranging from 0-1 like this:

<br>

<img src="https://drive.google.com/uc?export=view&id=1y0MIUVDW9kwOeDItmGdK9LZL8vFumoCO" width=700/>

<br>
<br>

The discriminator is a Convolutional Neural Network (CNN) using convolutional layers. A convolution works like this (animation by Michael Plotke taken from [here](https://commons.wikimedia.org/wiki/File:3D_Convolution_Animation.gif)):

<img src="https://drive.google.com/uc?export=view&id=1CijsOj0SLCNHy0ZQVw7Y0qnawRwFcN1N" width=400/>



We can calculate how a convolution changes its input with the following formula:

$\mathbf O = (\frac{I - K + 2*P}{S}) + 1$
- O = Output dimensions
- I = Input dimensions
- K = Kernel size (size of the filter, 3x3 pixels in the animation)
- P = Padding (amount of padding with zeros around the image, 1 pixel in the animation)
- S = Stride (by how many pixels we move the kernel, 1 pixel in the animation)

##### **Task 1: Specify the discriminator model**

The discriminator should have the following layers:
1. **Block 1** 
- *Convolution 1*
  - Channels input: ?
  - Channels output: 64
  - Kernel size: 4x4
  - Stride: 2
  - Padding: 1
  - No Bias
- *Leaky ReLU*
  - negative slope of 0.2

2. **Block 2**
- *Convolution 2*
  - Channels input: 64
  - Channels output: 128
  - Other parameters as in the previous one
- *Batch Normalization*
  - number of features: 128
- *Leaky ReLU*
  - negative slope of 0.2

3. **Block 3**
- *Convolution 3*
  - Channels input: 128
  - Channels output: 256
  - Other parameters as in the previous one
- *Batch Normalization*
  - number of features: 256
- *Leaky ReLU*
  - negative slope of 0.2

4. **Block 4**
- *Convolution 4*
  - Channels input: 256
  - Channels output: 512
  - Other parameters as in the previous one
- *Batch Normalization*
  - number of features: 512
- *Leaky ReLU*
  - negative slope of 0.2

5. **Block 5**
- *Convolution 5*
  - Channels input: 512
  - Channels output: 1
  - Kernel size: 4x4
  - Stride: ?
  - Padding: 0
  - No Bias
- *Sigmoid*

Please decide how many channels we put into the first convolutional layer and what stride we need in order to reach an output of nCx1x1 (using the formula above).

If you get stuck on how to implement the layers, you can take a look [here](https://colab.research.google.com/github/Mathis1993/DCGAN/blob/master/Specify_Models_Pytorch.ipynb).

To look up how to specify a certain layer, you can have a look in the [documentation](https://pytorch.org/docs/stable/nn.html).

In [0]:
class Discriminator(nn.Module):

  def __init__(self):
    super(Discriminator, self).__init__()
    
    ##Your code here

  def forward(self, x):
    
    ##Your code here

    return x

#### Generator

The generator goes the other way round: It takes as input a random vector from latent space and upsamples it to be an image of 3x64x64, like this:

<br>

<image src="https://drive.google.com/uc?export=view&id=1R6krNT2BMjeFctIX3Mok4zOyeglt9HXK" width=700>

<br>
<br>
<br>

Where we went from a whole image to one value in the discriminator, we now want to go from a point in latent space to a whole image. To do this, we can use transposed convolutions, which up-sample our input.

In a regular convolution operation, we would eg use a 2x2 kernel and apply it to a 3x3 image (without padding and a stride of 1) resulting in a 2x2 feature map. Mathematically, we change the kernel into a sparse matrix and flatten the image. Then, we can multiply both matrices and get the flattened feature map, which we can reshape to a 2x2 output.

<br>

$$\begin{array}{|c|c|}
\hline w_{00} & w_{01} \\\hline
  w_{10} & w_{11} \\\hline
\end{array} 
conv. 
\begin{array}{|c|c|c|}
\hline x_{0} & x_{1} & x_{2} \\\hline
  x_{3} & x_{4} & x_{5} \\\hline
  x_{6} & x_{7} & x_{8} \\\hline
\end{array}
=
\begin{array}{|c|c|}
\hline y_{0} & y_{1} \\\hline
  y_{2} & y_{3} \\\hline
\end{array}  
$$

$$\begin{pmatrix} 
w_{00} & w_{01} & 0 & w_{10} & w_{11} & 0 & 0 & 0 & 0 \\ 
0 & w_{00} & w_{01} & 0 & w_{10} & w_{11} & 0 & 0 & 0 \\
0 & 0 & 0 & w_{00} & w_{01} & 0 & w_{10} & w_{11} & 0 \\
0 & 0 & 0 & 0 & w_{00} & w_{01} & 0 & w_{10} & w_{11} 
\end{pmatrix} 
* 
\begin{pmatrix} 
x_{0} \\ x_{1} \\ x_{2} \\ x_{3} \\ x_{4} \\ x_{5} \\ x_{6} \\ x_{7} \\ x_{8}\end{pmatrix} 
= 
\begin{pmatrix} 
y_{0} \\ y_{1} \\ y_{2} \\ y_{3} \\ y_{4} 
\end{pmatrix}$$

<br>

When we use a transposed convolution, we transpose the sparse kernel matrix and multiply it a flattened input, yielding an up-sampled output. 

<br>

$$\begin{pmatrix}
w_{00} & 0 & 0 & 0 \\
w_{01} & w_{00} & 0 & 0 \\
0 & w_{01} & 0 & 0 \\
w_{10} & 0 & w_{00} & 0 \\
w_{11} & w_{10} & w_{01} & w_{00} \\
0 & w_{11} & 0 & w_{01} \\
0 & 0 & w_{10} & 0 \\
0 & 0 & w_{11} & w_{10} \\
0 & 0 & 0 & w_{11}
\end{pmatrix}
*
\begin{pmatrix} 
a_{0} \\ a_{1} \\ a_{2} \\ a_{3} \\ a_{4} 
\end{pmatrix}
=
\begin{pmatrix} 
b_{0} \\ b_{1} \\ b_{2} \\ b_{3} \\ b_{4} \\ b_{5} \\ b_{6} \\ b_{7} \\ b_{8}\end{pmatrix} 
$$ 
<br>

In [0]:
class Generator(nn.Module):

  def __init__(self):
    super(Generator, self).__init__()
    
    self.convT_block1 = nn.Sequential(
        #100 x 1 x 1
        nn.ConvTranspose2d(in_channels=100, out_channels=512, kernel_size=4, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.convT_block2 = nn.Sequential(
        #512 x 4 x 4
        nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )

    self.convT_block3 = nn.Sequential(
        #256 x 8 x 8
        nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )

    self.convT_block4 = nn.Sequential(
        #128 x 16 x 16
        nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )

    self.convT_block5 = nn.Sequential(
        #64 x 32 x 32
        nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False),
        nn.Tanh()
    )
        #3 x 64 x 64

  def forward(self, x):
    x = self.convT_block1(x)
    x = self.convT_block2(x)
    x = self.convT_block3(x)
    x = self.convT_block4(x)
    x = self.convT_block5(x)
    return x

#### Instantiate Models

This will also give an overview of the models.

Discriminator

In [0]:
discriminator = Discriminator().to(device)

#initialize weights
discriminator.apply(weights_init)

Generator

In [0]:
generator = Generator().to(device)

#initialize weights
generator.apply(weights_init)

### Loss Function

We are using Binary Cross Entropy Loss here.

In [0]:
criterion = nn.BCELoss()

### Optimizers

Adam optimizers with best-practice parameters.

In [0]:
optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))

### Training

Let's loop over our dataset a couple of times. Each time we train the discriminator on a batch of real and a batch of fake data. The generator is trained by using the discriminator's output when fed fake data.

##### **Task 2: Training Loop**

- Calculate the actual batch size

- Train the discriminator on a real batch of data
1. Zero the gradient buffer
2. Forward pass: Pass a batch of real images through the discriminator
3. Calculate the loss
4. Backward pass: Call the backpropagation on the loss to update the model weights

- Train the generator
1. Zero the gradient buffer
2. Forward pass: Pass a batch of fake images through the discriminator
3. Calculate the loss
4. Backward pass: Call the backpropagation on the loss to update the model

If you can't figure out the solution to some thing, you can always look at the corresponding section in this [notebook](https://colab.research.google.com/github/Mathis1993/DCGAN/blob/master/DCGAN_collapsed_view.ipynb) containing full code.

In [0]:
#To track how the latent space is given sense
fixed_noise = torch.randn(64, 100, 1, 1, device=device)

#To track progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
#For each epoch
for epoch in range(num_epochs):
    
    #For each batch in the dataloader
    for i, data in enumerate(dataloader, 0): #at what index to start

      #extract real images from list holding data and nonsense targets
      #and push to device
      data_real = data[0].to(device)

      #calculate acutal batch size (in the last batch per epoch, it could be != our determined batch_size)
      b_size = ##Your code here

      #targets
      label_real = torch.full(size=(b_size,), fill_value=1).to(device)
      label_fake = torch.full(size=(b_size,), fill_value=0).to(device)

      #train discriminator on real batch
      
      #zero gradient buffer
      ##Your code here

      #forward pass (Consider the output is of dimensions (n_batch, 1, 1, 1), so we have to collapse it to (n_batch))
      output_d_real = ##Your code here
      
      #calculate loss (using our loss-function, the output and adequate labels)
      loss_d_real = ##Your code here
      
      #backward pass (call the backpropagation on the loss)
      ##Your code here

      #train discriminator on fake batch
      #Generate batch of latent vectors
      z = torch.randn(b_size, 100, 1, 1, device=device)
      #Generate fake image batch with the generator
      data_fake = generator(z)
      output_d_fake = discriminator(data_fake.detach()).view(-1) #detach fake images from generator
      loss_d_fake = criterion(output_d_fake, label_fake)
      loss_d_fake.backward()

      #summarize loss 
      loss_d = loss_d_real + loss_d_fake
      #update discriminator weights
      optimizerD.step()

      #train generator
      
      #zero gradient buffer
      ##Your code here

      #as we just updated the discriminator, pass the fake data through it again
      #remember to collapse the output
      output_g = ##Your code here
      
      #calculate loss (using our loss-function, the output and adequate labels)
      loss_g = ##Your code here
      
      #backward pass (call the backpropagation on the loss)
      ##Your code here
      
      #update generator weights
      optimizerG.step()

      #track losses
      G_losses.append(loss_d.item())
      D_losses.append(loss_g.item())

      #How did D and G perform in discriminating/generating images?
      D_x = output_d_real.mean().item() * 100
      D_G_z1 = output_d_fake.mean().item() * 100
      D_G_z2 = output_g.mean().item() * 100

      #print information
      if (iters % 20 == 0):
        print("Epoch [{:02d}/{:02d}], Batch [{:02d}/{:02d}], Iteration {:04d}, D_loss: {:06.3f}, G_loss: {:06.3f}. Real images recognized as real: {:05.2f}%. Fake images fooled by: {:05.2f}% and {:05.2f}%.".format(epoch+1, num_epochs, i+1, len(dataloader), iters+1, loss_d.item(), loss_g.item(), D_x, D_G_z1, D_G_z2))

      #save generated images
      if (iters % 40 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = generator(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

      iters += 1

### Evaluation

Plot the discriminator's and generator's training losses over the iterations.

In [0]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

Let's take a look at a batch of real and a batch of generated images. The generated images were generated in the last training epoch.

In [0]:
#Get batch of images
real_batch = next(iter(dataloader))

#Plot real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

#Plot fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1].cpu(),(1,2,0)))
plt.show()

Generate some more images by feeding points from latent space into the generator.

In [0]:
z = torch.randn(64, 100, 1, 1, device=device)
with torch.no_grad():
  fake = generator(z).detach().cpu()
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("More Fake Images")
plt.imshow(np.transpose(vutils.make_grid(fake, padding=2, normalize=True),(1,2,0)))

Let's visualize the progress our GAN made in a nice animation (This one is from from training a bit longer).

<img src="https://drive.google.com/uc?export=view&id=1KAilGmvF22un85-XQMgSr7GhlHugJFFJ" width=700/>

Code to create your own animation:

In [0]:
#Animation in output is so big that colab takes too long to save, so don't output it right now
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)
#HTML(ani.to_jshtml())

If you want to take a look at how well a GAN for faces can do, check out this [website](https://thispersondoesnotexist.com/).